In [1]:
import pandas as pd  # keywords are highlighted in green, other strings in red, etc.
import numpy as np
import statistics as stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import StandardScaler

In [2]:
file = pd.read_csv('fifa21_train.csv')
file.shape

(11701, 101)

In [3]:
file.head()


,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [4]:
def clean_data(df):
    df.columns=list(map(lambda x: x.replace(" ","_"), df.columns))
    df=df.drop(['ID', 'Name', 'Nationality', 'Joined','Contract', 'Team_&_Contract', 'Loan_Date_End', 'Position'], axis=1 )
    df=df.dropna()
    df = df.drop_duplicates()
   
    return df

In [5]:
file= clean_data(file)

In [6]:
file.head()

,Age,Club,BP,Height,Weight,foot,Growth,Value,Wage,Release_Clause,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,26,FC Lausanne-Sport,CM,"5'9""",161lbs,Right,1,€525K,€4K,€801K,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,30,Beijing Sinobo Guoan FC,ST,"6'0""",159lbs,Right,0,€8.5M,€23K,€0,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,33,Al Hilal,CAM,"5'4""",134lbs,Right,0,€9M,€49K,€15.3M,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,22,Swansea City,CDM,"5'10""",152lbs,Right,13,€275K,€4K,€694K,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,23,US Orléans Loiret Football,CDM,"5'11""",150lbs,Right,8,€725K,€2K,€1.4M,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [7]:
def separe_num_cat(df):
    df.numerical = df._get_numeric_data()
    df.categoricals= df.select_dtypes(['object'])
    
    return df


In [8]:
file= separe_num_cat(file)

C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/4095467325.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.numerical = df._get_numeric_data()
C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/4095467325.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.categoricals= df.select_dtypes(['object'])


In [9]:
file.categoricals.head()

,Club,BP,Height,Weight,foot,Value,Wage,Release_Clause,W/F,SM,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,FC Lausanne-Sport,CM,"5'9""",161lbs,Right,€525K,€4K,€801K,4 ★,2★,...,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1
1,Beijing Sinobo Guoan FC,ST,"6'0""",159lbs,Right,€8.5M,€23K,€0,3 ★,4★,...,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2
2,Al Hilal,CAM,"5'4""",134lbs,Right,€9M,€49K,€15.3M,4 ★,4★,...,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2
3,Swansea City,CDM,"5'10""",152lbs,Right,€275K,€4K,€694K,2 ★,2★,...,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2
4,US Orléans Loiret Football,CDM,"5'11""",150lbs,Right,€725K,€2K,€1.4M,2 ★,3★,...,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2


In [10]:
def specialchar_remove(df):
    specialchars = ['€','★',"lbs"]
    for i in df.categoricals:
            df.categoricals[i] = df.categoricals[i].replace(specialchars,'', regex=True)
    return df

In [11]:
file= specialchar_remove(file)

C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/3440074644.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.categoricals[i] = df.categoricals[i].replace(specialchars,'', regex=True)


In [12]:
file.categoricals.head()

,Club,BP,Height,Weight,foot,Value,Wage,Release_Clause,W/F,SM,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,FC Lausanne-Sport,CM,"5'9""",161,Right,525K,4K,801K,4,2,...,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1
1,Beijing Sinobo Guoan FC,ST,"6'0""",159,Right,8.5M,23K,0,3,4,...,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2
2,Al Hilal,CAM,"5'4""",134,Right,9M,49K,15.3M,4,4,...,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2
3,Swansea City,CDM,"5'10""",152,Right,275K,4K,694K,2,2,...,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2
4,US Orléans Loiret Football,CDM,"5'11""",150,Right,725K,2K,1.4M,2,3,...,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2


In [13]:
def split_data(df):
    for i in df.categoricals:
        
        if [df.categoricals[i].isin(['+-'])]:
            df.categoricals[i] = df.categoricals[i].str.split('+').str[0]
            
        elif [df.categoricals[i].isin((['-']))]:
            df.categoricals[i] = df.categoricals[i].str.split('-').str[0]

        else: 
            df.categoricals[i] = df.categoricals[i].str.split('+').str[0]
    return df
            
    
    
    

In [14]:
file= split_data(file)


C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/3313704691.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.categoricals[i] = df.categoricals[i].str.split('+').str[0]


In [15]:
file.categoricals.head()

,Club,BP,Height,Weight,foot,Value,Wage,Release_Clause,W/F,SM,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,FC Lausanne-Sport,CM,"5'9""",161,Right,525K,4K,801K,4,2,...,59,59,59,59,58,54,54,54,58,15
1,Beijing Sinobo Guoan FC,ST,"6'0""",159,Right,8.5M,23K,0,3,4,...,53,53,53,57,53,48,48,48,53,18
2,Al Hilal,CAM,"5'4""",134,Right,9M,49K,15.3M,4,4,...,56,56,56,59,53,41,41,41,53,12
3,Swansea City,CDM,"5'10""",152,Right,275K,4K,694K,2,2,...,58,58,58,56,57,58,58,58,57,14
4,US Orléans Loiret Football,CDM,"5'11""",150,Right,725K,2K,1.4M,2,3,...,64,64,64,64,63,61,61,61,63,15


In [16]:
def k_m_remove(df):
    df.categoricals['Release_Clause'] = df.categoricals['Release_Clause'].replace({'€': '', 'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(float)
    df.categoricals['Wage'] = df.categoricals['Wage'].replace({'€': '', 'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(float)
    df.categoricals['Value'] = df.categoricals['Value'].replace({'€': '', 'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(float)
    df.categoricals['Hits'] = df.categoricals['Hits'].replace({'€': '', 'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(float)
    return df

In [17]:
file= k_m_remove(file)

C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/4279060786.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.categoricals['Release_Clause'] = df.categoricals['Release_Clause'].replace({'€': '', 'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(float)
C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/4279060786.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.categoricals['Wage'] = df.categoricals['Wage'].replace({'€': '', 'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(float

In [18]:
file.categoricals.head()

,Club,BP,Height,Weight,foot,Value,Wage,Release_Clause,W/F,SM,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,FC Lausanne-Sport,CM,"5'9""",161,Right,525000.0,4000.0,801000.0,4,2,...,59,59,59,59,58,54,54,54,58,15
1,Beijing Sinobo Guoan FC,ST,"6'0""",159,Right,8500000.0,23000.0,0.0,3,4,...,53,53,53,57,53,48,48,48,53,18
2,Al Hilal,CAM,"5'4""",134,Right,9000000.0,49000.0,15300000.0,4,4,...,56,56,56,59,53,41,41,41,53,12
3,Swansea City,CDM,"5'10""",152,Right,275000.0,4000.0,694000.0,2,2,...,58,58,58,56,57,58,58,58,57,14
4,US Orléans Loiret Football,CDM,"5'11""",150,Right,725000.0,2000.0,1400000.0,2,3,...,64,64,64,64,63,61,61,61,63,15


In [19]:
def clean_Height(df):
    df.H_feet = df.categoricals['Height'].str.split("'").str[0]
    df.H_inch = df.categoricals['Height'].str.split("'").str[1].str.split("\"").str[0]
    df.categoricals['Height'] = (df.H_feet.astype(float) * 12) + df.H_inch.astype(float)
    
    return df

In [20]:
file= clean_Height(file)

C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/3807522444.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.H_feet = df.categoricals['Height'].str.split("'").str[0]
C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/3807522444.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.H_inch = df.categoricals['Height'].str.split("'").str[1].str.split("\"").str[0]
C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/3807522444.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.categoricals['Height'] = (df.H_fe

In [21]:
file.categoricals.head()

,Club,BP,Height,Weight,foot,Value,Wage,Release_Clause,W/F,SM,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,FC Lausanne-Sport,CM,69.0,161,Right,525000.0,4000.0,801000.0,4,2,...,59,59,59,59,58,54,54,54,58,15
1,Beijing Sinobo Guoan FC,ST,72.0,159,Right,8500000.0,23000.0,0.0,3,4,...,53,53,53,57,53,48,48,48,53,18
2,Al Hilal,CAM,64.0,134,Right,9000000.0,49000.0,15300000.0,4,4,...,56,56,56,59,53,41,41,41,53,12
3,Swansea City,CDM,70.0,152,Right,275000.0,4000.0,694000.0,2,2,...,58,58,58,56,57,58,58,58,57,14
4,US Orléans Loiret Football,CDM,71.0,150,Right,725000.0,2000.0,1400000.0,2,3,...,64,64,64,64,63,61,61,61,63,15


In [22]:
def changetonumeric(df):
    df.col_to_numercic = ['Weight', 'Value', 'Wage', 'Release_Clause', 'W/F', 'SM', 'IR', 'Hits', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK']
    for i in range(len(df.col_to_numercic)):
        df.categoricals[df.col_to_numercic[i]] =  pd.to_numeric(df.categoricals[df.col_to_numercic[i]])
    return df

In [23]:
file = changetonumeric(file)

C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/851047728.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.col_to_numercic = ['Weight', 'Value', 'Wage', 'Release_Clause', 'W/F', 'SM', 'IR', 'Hits', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK']
C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/851047728.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.categoricals[df.col_to_numercic[i]] =  pd.to_numeric(df.categoricals[df.col_to_numercic[i]])


In [24]:
file.categoricals.dtypes

Club               object
BP                 object
Height            float64
Weight              int64
foot               object
Value             float64
Wage              float64
Release_Clause    float64
W/F                 int64
SM                  int64
A/W                object
D/W                object
IR                  int64
Hits              float64
LS                  int64
ST                  int64
RS                  int64
LW                  int64
LF                  int64
CF                  int64
RF                  int64
RW                  int64
LAM                 int64
CAM                 int64
RAM                 int64
LM                  int64
LCM                 int64
CM                  int64
RCM                 int64
RM                  int64
LWB                 int64
LDM                 int64
CDM                 int64
RDM                 int64
RWB                 int64
LB                  int64
LCB                 int64
CB                  int64
RCB         

In [25]:
def concat_separe(df):
    df = pd.concat([df.numerical, df.categoricals], axis=1)
    df.numerical = df._get_numeric_data()
    df.categoricals= df.select_dtypes(['object'])
    df.y = df.numerical['OVA']
    df.X = df.numerical.drop(['OVA'],axis=1)
    df.categoricals = df.categoricals.drop(['Club', 'BP'], axis=1)

    return df

In [26]:
file= concat_separe(file)

C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/2451539406.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.numerical = df._get_numeric_data()
C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/2451539406.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.categoricals= df.select_dtypes(['object'])
C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/2451539406.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.y = df.numerical['OVA']
C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/2451539406.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/s

In [27]:
file.categoricals.head()

,foot,A/W,D/W
0,Right,High,Medium
1,Right,High,Low
2,Right,High,Medium
3,Right,Medium,Medium
4,Right,Low,Medium


In [28]:
def onehotencoder(df):
    df.hot_encoder = OneHotEncoder().fit(df.categoricals)
    df.hot_encoded = df.hot_encoder.transform(df.categoricals).toarray()
    df.cols = [colname for row in df.hot_encoder.categories_ for colname in row]
    df.cols
    df.hot_encoded = pd.DataFrame(df.hot_encoded,columns=df.cols)
    df.cols_to_drop = [row[0] for row in df.hot_encoder.categories_]
    df.hot_encoded = df.hot_encoded.drop(df.cols_to_drop, axis=1)
  

    
    return df

In [29]:
file= onehotencoder(file)

C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/3684955578.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.hot_encoded = df.hot_encoder.transform(df.categoricals).toarray()
C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/3684955578.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cols = [colname for row in df.hot_encoder.categories_ for colname in row]
C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/3684955578.py:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cols_to_drop = [row[0] for row in df.hot_encoder.categories_]


In [30]:
file.hot_encoded

,Right,Low,Medium,Low,Medium
0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,1.0
4,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...
11406,1.0,0.0,0.0,1.0,0.0
11407,0.0,0.0,1.0,0.0,1.0
11408,1.0,0.0,1.0,0.0,1.0
11409,1.0,0.0,0.0,0.0,1.0


In [31]:
def normilizing(df):
    df.transformer = MinMaxScaler().fit(df.X)
    df.x_normalized = df.transformer.transform(df.X)
    df.x_normalized=pd.DataFrame(df.x_normalized, columns=df.X.columns)
    
    return df

In [32]:
file= normilizing(file)

C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/1904431323.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.x_normalized = df.transformer.transform(df.X)


In [33]:
file.x_normalized

,Age,Growth,Attacking,Crossing,Finishing,Heading_Accuracy,Short_Passing,Volleys,Skill,Dribbling,...,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK
0,0.370370,0.038462,0.546835,0.545455,0.478261,0.431818,0.710843,0.465116,0.569087,0.615385,...,0.614286,0.614286,0.614286,0.642857,0.637681,0.527778,0.527778,0.527778,0.637681,0.075949
1,0.518519,0.000000,0.817722,0.681818,0.826087,0.806818,0.686747,0.837209,0.777518,0.857143,...,0.528571,0.528571,0.528571,0.614286,0.565217,0.444444,0.444444,0.444444,0.565217,0.113924
2,0.629630,0.000000,0.744304,0.761364,0.793478,0.329545,0.807229,0.825581,0.892272,0.879121,...,0.571429,0.571429,0.571429,0.642857,0.565217,0.347222,0.347222,0.347222,0.565217,0.037975
3,0.222222,0.500000,0.506329,0.431818,0.423913,0.602273,0.614458,0.372093,0.505855,0.538462,...,0.600000,0.600000,0.600000,0.600000,0.623188,0.583333,0.583333,0.583333,0.623188,0.063291
4,0.259259,0.307692,0.524051,0.488636,0.369565,0.636364,0.686747,0.348837,0.555035,0.648352,...,0.685714,0.685714,0.685714,0.714286,0.710145,0.625000,0.625000,0.625000,0.710145,0.075949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11406,0.481481,0.000000,0.627848,0.340909,0.750000,0.681818,0.554217,0.651163,0.562061,0.626374,...,0.414286,0.414286,0.414286,0.457143,0.434783,0.375000,0.375000,0.375000,0.434783,0.126582
11407,0.333333,0.192308,0.460759,0.636364,0.228261,0.522727,0.578313,0.279070,0.480094,0.571429,...,0.628571,0.628571,0.628571,0.700000,0.710145,0.625000,0.625000,0.625000,0.710145,0.088608
11408,0.407407,0.153846,0.106329,0.068182,0.119565,0.090909,0.216867,0.139535,0.124122,0.131868,...,0.142857,0.142857,0.142857,0.157143,0.144928,0.138889,0.138889,0.138889,0.144928,0.759494
11409,0.222222,0.307692,0.617722,0.659091,0.684783,0.522727,0.638554,0.430233,0.580796,0.725275,...,0.400000,0.400000,0.400000,0.500000,0.449275,0.263889,0.263889,0.263889,0.449275,0.101266


In [34]:
def concathotnorm(df):
    df.X_concat = pd.concat([df.x_normalized, df.hot_encoded], axis=1)
    
    return df

In [35]:
file= concathotnorm(file)

C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/1254174447.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.X_concat = pd.concat([df.x_normalized, df.hot_encoded], axis=1)


In [36]:
file.X_concat.shape

(11411, 92)

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(file.X_concat, file.y, test_size=0.2, random_state=42)

In [38]:
from sklearn import linear_model
#we train/fit our model like yesterday
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [39]:
from sklearn.metrics import r2_score
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

0.9136225059991091

In [40]:
predictions

array([73.00431919, 62.73602772, 77.11179447, ..., 54.3551054 ,
       74.61620617, 69.28729534])

In [41]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import math
predictions_test = lm.predict(X_test)
r2 = r2_score(y_test, predictions_test)
mse = mean_squared_error(y_test, predictions_test)
mae = mean_absolute_error(y_test, predictions_test)
rmse = math.sqrt(mse)
print("r2 score is:", r2)
print("mse is:", mse)
print("mae is:",mae)
print("rmse is:",rmse)

r2 score is: 0.9128002655074701
mse is: 4.200867961841421
mae is: 1.58762891471151
rmse is: 2.0496019032586354


In [42]:
data_for_p = pd.read_csv('fifa21_validate.csv')

In [43]:
data_for_p= clean_data(data_for_p)
data_for_p= separe_num_cat(data_for_p)
data_for_p= specialchar_remove(data_for_p)
data_for_p= split_data(data_for_p)
data_for_p= k_m_remove(data_for_p)
data_for_p= clean_Height(data_for_p)
data_for_p = changetonumeric(data_for_p)
data_for_p= concat_separe(data_for_p)
data_for_p= onehotencoder(data_for_p)
data_for_p= normilizing(data_for_p)
data_for_p= concathotnorm(data_for_p)



C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/4095467325.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.numerical = df._get_numeric_data()
C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/4095467325.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.categoricals= df.select_dtypes(['object'])
C:\Users\gonza\AppData\Local\Temp/ipykernel_51212/3440074644.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.categoricals[i] = df.categoricals[i].replace(specialchars,'', regex=True)
C:\Users\gonza\App

In [44]:
data_for_p.X_concat.dtypes


Age          float64
Growth       float64
Attacking    float64
Crossing     float64
Finishing    float64
              ...   
Right        float64
Low          float64
Medium       float64
Low          float64
Medium       float64
Length: 92, dtype: object

In [45]:
results_for_p = lm.predict(data_for_p.X_concat)

In [46]:
data_for_p2 = pd.read_csv('fifa21_validate.csv')

In [47]:
pd.concat([data_for_p2, pd.Series(results_for_p)],axis=1).head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA,0
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,"FK Austria Wien Jun 30, 2021 On Loan","6'2""",194lbs,...,61+2,59+2,61+2,67+2,67+2,67+2,61+2,16+2,67,4.844263e+09
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,LM CAM,Heart of Midlothian 2020 ~ 2020,"5'8""",154lbs,...,47+2,49+2,45+2,38+2,38+2,38+2,45+2,17+2,68,6.528231e+09
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,GK,Real Salt Lake 2018 ~ 2020,"6'2""",176lbs,...,18+2,16+2,16+2,18+2,18+2,18+2,16+2,53+2,54,-7.738869e+08
3,255120,N. Kenneh,16,England,Leeds United,CDM,CB CDM RB,Leeds United 2020 ~ 2022,"6'3""",170lbs,...,54+2,53+2,53+2,54+2,54+2,54+2,53+2,11+2,55,4.276134e+09
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,CM CDM,1. FSV Mainz 05 2019 ~ 2023,"6'2""",170lbs,...,72+2,70+2,69+2,68+2,68+2,68+2,69+2,18+2,70,6.497918e+09


In [48]:
r2_2 = r2_score(data_for_p.y, results_for_p)
print(r2_2)

-7.733901704607593e+17
